# Exploratory Analysis of SEIR-QD Model

LM, last updated: 3/27

In [21]:
import pandas as pd
import numpy as np
import scipy.integrate

import bokeh.io
import bokeh.application
import bokeh.application.handlers
import bokeh.models
import bokeh.plotting

bokeh.io.output_notebook()

Loading BokehJS ...

This notebook is meant to get a feel for the SEIR-QD model. It is still being developed.

# INTERACTIVE ODEINT SEIR-QD

This section followed the interactive notebook made by Justin Bois and Michael Elowitz at Caltech: http://be150.caltech.edu/2019/handouts/02_intro_to_python_for_biological_circuits.html

This model is outlined in *Rational evaluation of various epidemic models based on the COVID-19 data of China* (https://www.medrxiv.org/content/10.1101/2020.03.12.20034595v1.full.pdf), and turned out to be one of the better models in terms of early-stage data collection. It takes into effect quarantine and self-protection. The differential equations can be found on page 8 of the supplement: https://www.medrxiv.org/content/medrxiv/suppl/2020/03/16/2020.03.12.20034595.DC1/2020.03.12.20034595-1.pdf. A more in depth look can be found here: https://arxiv.org/pdf/2002.06563.pdf

Here, I am just testing the basics of the model. I have pulled some reasonable parameters, but later on we can play with the parameters a bit.

### Reservoirs

**S, Susceptible**: Have not been infected. For Lombardi, $10^7$

**E, Exposed**: Infected but not infectious, patient in latent period

**I, Infectious**: Infected and not quarantined, can infect others.

**R, Recovered**: Recovered, not infectious, not susceptible.

**Q, Quarantined**: Infected and quarantined (not infectious).

**D, Death**: Passed, no longer infectious.

### Parameters

Here we define the regions we want to toggle our parameters on. I am still looking into this, but here are some preliminary thoughts:

**$\beta$ = infection rate**, from earlier plotting, $10^{(-8)} - 10^{(-6)}$ seem reasonable. I use $log_{10}$ to make the slider comprehensible

**$\delta$ = recovery rate**, which we think is on the order of 10-40 days.

**$\gamma$ = transition of exposed individuals to infected**, which we aren't sure of, especially with the unknown number of asymptomatics. We will leave this open from 0 to 1.

**$\alpha$ = protection rate of susceptible individuals**, which we also don't know, and is most likely dynamic over the course of the outbreak. We will leave this open from 0 to 1 as well.

**$\lambda$ = transition rate of infected to quarantined with infection**, same as above.

**$\kappa$ = death rate**, which we think is around 0.01-0.06. We will leave a range between 0.01 and 0.1.

In [22]:
days = 60

# Time points we want for the solution
t = np.arange(0, days)

# Initial condition, 1000 exposed, 10**7 suscept.
yz_0 = np.array([10**7, 1000, 0, 0, 0, 0, 0])

In [23]:
def seirqd(dat, t, beta, delta, gamma, alpha, lambda_, kappa):
    s = dat[0]
    e = dat[1]
    i = dat[2]
    q = dat[3]
    r = dat[4]
    d = dat[5]
    sa = dat[6]
    
    dsdt = - beta * s * i - alpha * s
    dedt = beta * s * i - gamma * e
    didt = gamma * e - lambda_ * i
    dqdt = lambda_ * i - delta * q - kappa * q
    drdt = delta * q
    dddt = kappa * q
    dsadt = alpha * s
    
    return [dsdt, dedt, didt, dqdt, drdt, dddt, dsadt]

In [24]:
class AttributeContainer(object):
    def __init__(self, **kw):
        self.__dict__ = kw

# Parameters for each slider
slider_params = (AttributeContainer(title='log 10 𝛽: infection rate', start= -8, end= -6, value=-7.15, step=0.001),
                 AttributeContainer(title='𝛿: recovery rate', start=1/40, end=1/10, value=0.04, step=0.001),
                 AttributeContainer(title='𝛾: exposed to infectious', start=0, end=1, value=0.32, step=0.001),
                 AttributeContainer(title='𝛼: protection rate', start=0, end=1, value=0.05, step=0.001),
                 AttributeContainer(title='𝜆: infected to quarantined with infection', start=0, end=1, value=0.09, step=0.001),
                 AttributeContainer(title='𝜅: death rate', start=0, end=0.1, value=0.03, step=0.001))

In [25]:
extra_args = (seirqd, yz_0, days)

def cascade_callback(source, x_range, y_range, sliders, seirqd, yz_0, n):
    # Set up time values, keeping minimum at zero
    t = np.linspace(0, x_range.end, n)
    
    # Convert logarithmic sliders
    slider_args = tuple(10**slider.value if 'log' in slider.title else slider.value 
                             for slider in sliders)

    # Integrate ODES
    yz = scipy.integrate.odeint(seirqd, yz_0, t, args=slider_args)

    # Update data source
    s, e, i, q, r, d, sa = yz.transpose()
    
    source.data['t'] = t
    source.data['r_vals'] = r
    source.data['i_vals'] = i
    source.data['d_vals'] = d
    source.data['q_vals'] = q
    source.data['e_vals'] = e
    source.data['sa_vals'] = sa

In [26]:
def cascade_plot(callback, sliders, extra_args):
    # Set up plot
    p = bokeh.plotting.figure(plot_width=600,
                              plot_height=400,
                              x_axis_label='time (days)',
                              y_axis_label='# individuals',
                              title = 'SEIR-QD tunable model')

    # Set up empty data source
    source = bokeh.models.ColumnDataSource()

    # Populate glyphs
    p.line('t', 'e_vals', source=source, line_width=2, color='orange', legend='E: infected but not infectious')
    p.line('t', 'i_vals', source=source, line_width=2, color='red', legend='I: infected, not quarantined')
    p.line('t', 'q_vals', source=source, line_width=2, color='purple', legend='Q: infected quarantined')
    p.line('t', 'r_vals', source=source, line_width=2, color='green', legend='R: recovered')
    p.line('t', 'd_vals', source=source, line_width=2, color='black', legend='D: death')
    #p.line('t', 'sa_vals', source=source, line_width=2, color='teal', legend='insusceptible')

    # Place the legend
    p.legend.location = 'top_left'

    # Update data according to callback
    callback(source, bokeh.models.Range1d(0, 60), None, 
             sliders, *extra_args)


    return p, source

In [27]:
def interactive_xy_plot(base_plot, callback, slider_params, extra_args):
    """Build an interactive x-y plot app."""
    
    def _plot_app(doc):
        # Build the initial plot and data source
        p, source = base_plot(callback, slider_params, extra_args)
        
        # Make sure axis ranges have no padding
        if type(p.x_range) == bokeh.models.ranges.Range1d:
            start, end = p.x_range.start, p.x_range.end
            p.x_range = bokeh.models.ranges.DataRange1d(p.x_range)
            p.x_range.start = start
            p.x_range.end = end
        if type(p.y_range) == bokeh.models.ranges.Range1d:
            start, end = p.y_range.start, p.y_range.end
            p.y_range = bokeh.models.ranges.DataRange1d(p.y_range)
            p.y_range.start = start
            p.y_range.end = end
        p.x_range.range_padding = 0
        p.y_range.range_padding = 0
        
        # Callbacks
        def _callback(attr, old, new):
            callback(source, p.x_range, p.y_range, sliders, 
                     *extra_args)
                
        # Set up sliders
        sliders = tuple(bokeh.models.Slider(start=param.start,
                                       end=param.end,
                                       value=param.value,
                                       step=param.step,
                                       title=param.title)
                            for param in slider_params)
        for slider in sliders:
            slider.on_change('value', _callback)
            
        # Execute callback upon changing axis values
        p.x_range.on_change('start', _callback)
        p.x_range.on_change('end', _callback)
        p.y_range.on_change('start', _callback)
        p.y_range.on_change('end', _callback)
        
        # Add the plot to the app
        widgets = bokeh.layouts.widgetbox(*sliders)
        doc.add_root(bokeh.layouts.column(widgets, p))

    handler = bokeh.application.handlers.FunctionHandler(_plot_app)
    return bokeh.application.Application(handler)

In [28]:
# Build the interactive plotting app
app = interactive_xy_plot(cascade_plot, cascade_callback, 
                          slider_params, extra_args)

# Show the app
notebook_url = 'localhost:8888'
bokeh.io.show(app, notebook_url=notebook_url)

ERROR:bokeh.server.protocol_handler:error handling message Message 'PATCH-DOC' (revision 1) content: {'events': [{'kind': 'ModelChanged', 'model': {'type': 'DataRange1d', 'id': '1008'}, 'attr': 'start', 'new': 0}], 'references': []}: TypeError("unsupported operand type(s) for *: 'NoneType' and 'float'")


# Computing Environment

In [29]:
%load_ext watermark
%watermark -v -p numpy,scipy,pandas,bokeh,holoviews,jupyterlab

CPython 3.7.5
IPython 7.13.0

numpy 1.18.1
scipy 1.4.1
pandas 1.0.1
bokeh 1.3.4
holoviews 1.12.5
jupyterlab 1.2.4
